INITIALIZE

In [4]:
import pyvisa
import numpy as np
import time
import matplotlib.pyplot as plt
import os
import csv
%matplotlib qt

In [5]:
def update_plot(fig, ax, line, lstVoltage, lstAbsCurrent):
    line.set_data(lstVoltage, lstAbsCurrent)
    ax.relim()
    ax.autoscale_view()
    fig.canvas.draw()
    fig.canvas.flush_events()

In [6]:

rm = pyvisa.ResourceManager()

In [7]:

m2461 = rm.open_resource('TCPIP0::132.181.53.57::inst0::INSTR')
m2461.timeout = 10000
m2461.write(f"reset()")

9

In [8]:
def fncMeasureOne(arrVMeasurement, \
                  device= 'pico',\
                  Sample = 'Test', sweepNum = 1,\
                  folderPath='230606\\Dev02\\Test',\
                  askToStart=False):
    if (device == 'pico'):
        fncMeasureOnePico(arrVMeasurement, Sample, sweepNum, folderPath, askToStart)
    elif (device == 'keith'):
        fncMeasureOneKL(arrVMeasurement, Sample, sweepNum, folderPath, askToStart)
    elif (device == 'keith1'):
        fncMeasureOneKL1(arrVMeasurement, Sample, sweepNum, folderPath, askToStart)
    elif (device == 'M2657A'):
        fncMeasureOneM2657A(arrVMeasurement, Sample, sweepNum, folderPath, askToStart)
    elif (device == 'M2461SWETSP'): #TSP means using TSP commands, SWE means sweep the whole thing from start to stop in one run
        fncMeasureOneM2461SWETSP(arrVMeasurement, Sample, sweepNum, folderPath, askToStart)
    elif (device == 'M2461POITSP'): #POI means set each point and then measure and move to another.
        fncMeasureOneM2461POITSP(arrVMeasurement, Sample, sweepNum, folderPath, askToStart)
    elif (device == 'M2461SWE'): #TSP means using TSP commands, SWE means sweep the whole thing from start to stop in one run
        fncMeasureOneM2461SWE(arrVMeasurement, Sample, sweepNum, folderPath, askToStart)
    else:
        print("Incorrect device name")

In [9]:
def fncMeasureMultiple(settings =['pico', [[0, 0.5, 0.1]]], \
                  Sample = 'Test',\
                  startNum = 1, \
                  folderPath='230606\\Dev02\\Test',\
                  askToStart=False): 
    sweepNum = 0
    dev, arrVMeasurements = settings
    for j in range (len(arrVMeasurements)):
        sweepNum = j + startNum
        fncMeasureOne(arrVMeasurements[j], dev , Sample, sweepNum, folderPath)
        time.sleep(5)

In [10]:
folderPath = '../Devices/250328_Fab241201_IrOxNewSM'
# Create a new directory
try:
    os.mkdir(folderPath)
except FileExistsError:
    print("Folder already exists")
    
folderPath = folderPath+ '\\Dev08'
# Create a new directory
try:
    os.mkdir(folderPath)
except FileExistsError:
    print("Folder already exists")

folderPath = folderPath+ '\\H02'
# Create a new directory
try:
    os.mkdir(folderPath)
except FileExistsError:
    print("Folder already exists")
sample = folderPath[-3:]

Folder already exists
Folder already exists
Folder already exists


In [ ]:
def setM2461IVMeasurementTSP (terminal = "FRONT",\
                  vRange = "AUTO",\
                  iRange = "AUTO",\
                  iCompliance = 7,\
                  iMeasMode = "4WIRE",\
                  iNPLC = 1,\
                  delay = 0.1): #(iRange = "AUTO", iLimit = 7, nplc = 1, delay = 0.1)
    # Reset and initialize instrutment
    m2461.write(f"defbuffer1.clear()")
    m2461.write(f"buffer.clearstats()")
    
    m2461.write(f"smu.reset()")
    m2461.write(f"status.reset()")
    m2461.write(f"errorqueue.clear()")
    m2461.write(f"dataqueue.clear()")
    
    
    ##############################################
    ## Make sure that no data left in the buffer
    # Send command to check the number of stored readings in defbuffer1
    m2461.write("print(defbuffer1.n)")
    # Read the response and convert it to an integer
    buffer_count = round(float(m2461.read().strip()))
    print(f"Number of data points in buffer: {buffer_count}")
    if buffer_count != 0:
        data = m2461.read()
        print(f"data remains in buffer: {data}")
    
    ##############################################
    m2461.write(f"smu.terminals = smu.TERMINALS_{terminal}" )

    # Setup current measurement mode
    m2461.write(f"smu.measure.sense = smu.SENSE_{iMeasMode}")

    #Set up current measurement range
    if iRange == "AUTO":
        m2461.write(f"smu.measure.autorange = smu.ON")
    else:
        m2461.write(f"smu.measure.autorange = smu.OFF")
        m2461.write(f"smu.measure.rangei = %f" % iRange)

    # Setup voltage source mode

    m2461.write(f"smu.source.func = smu.FUNC_DC_VOLTAGE")
    if vRange == "AUTO":
        m2461.write(f"smu.source.autorange = smu.ON")  
    else:
        m2461.write(f"smu.source.autorange = smu.OFF")
        m2461.write(f"smu.source.autorange = {vRange}") 
    
    #Set up current compliance
    m2461.write(f"smu.source.ilimit.level = %f" % iCompliance)

    #Set the measurement integration time and delay
    m2461.write(f"smu.measure.nplc = %f" % iNPLC)
    m2461.write(f"smu.source.delay = %f" % delay)

    m2461.write(f"beeper.beep(0.5,100)")
    return True
setM2461IVMeasurementTSP()   
  
    
    #m2461.write(f"display.changescreen(display.SCREEN_HOME)")

def measureM2461OnePointTSP(setV, outputON = False, resetV = True):
    #setM2461IVMeasurement()
    #m2461.write(f"bufData = buffer.make(1)") 
    m2461.write(f"smu.source.func = smu.FUNC_DC_VOLTAGE")
    m2461.write(f"smu.source.level = %f" %setV)
    
    if (not outputON):
        m2461.write(f"smu.source.output = 1")
    
    m2461.write(f"smu.measure.func = smu.FUNC_DC_VOLTAGE")
    m2461.write(f"print(smu.measure.read())") 
    voltage = m2461.read()

    m2461.write(f"smu.measure.func = smu.FUNC_DC_CURRENT")
    m2461.write(f"print(smu.measure.read())") 
    #m2461.write(f'printbuffer(1, 1, defbuffer1.sourcevalues, defbuffer1.readings)')
    current = m2461.read()
    #m2461.write(f"buffer.delete(bufData)")

    #Turn off the output
    if resetV:
        m2461.write(f"smu.source.level = 0")
        m2461.write(f"smu.source.output = 0")
    return [float(current), float(voltage)] #[float(arrRawData[i]) for i in range(len(arrRawData))]

# rawData
current, voltage = measureM2461OnePointTSP(2,False,True)
print ("Current: ", current, "Voltage: ", voltage)
#print (rawData)


In [12]:
def fncMeasureOneM2461POITSP(arrVMeasurement, \
                  Sample = 'Test', sweepNum = 1,\
                  folderPath='250319\\Dev02\\Test',\
                  askToStart=False,\
                  terminal = "FRONT",\
                  vRange = "AUTO",\
                  iRange = "AUTO",\
                  iCompliance = 7,\
                  iMeasMode = "2WIRE", \
                  iNPLC = 1, \
                  delay = 0.1,\
                  ):
    Vstart, Vstop, Vstep = arrVMeasurement
    fileName = "Sweep"+str(sweepNum)+Sample+'Star'+str(Vstart).replace("-","M")+'_Stop'+str(Vstop).replace("-","M")+'_Step'+str(Vstep).replace("-", "M")+'.csv'
    filePath = os.path.join(folderPath, fileName)
    print(filePath)
    stepNum = np.floor(abs((Vstart - Vstop)/Vstep)).astype(int) + 1

    setM2461IVMeasurementTSP(terminal, vRange, iRange, iCompliance, iMeasMode, iNPLC, delay)
    absVstart = abs(Vstart)
    absVstop = abs(Vstop)
    # Set up source range
    if abs(Vstart) > abs(Vstop): 
        m2461.write(f"smu.source.rangev = %f" % absVstart)
    else:
        m2461.write(f"smu.source.rangev = %f" % absVstop)
    
    lstCurrent = []
    lstVoltage = []
    lstAbsCurrent = []
    
    fig, ax = plt.subplots()
    line, = ax.plot([], [], 'o-')
    ax.set_xlabel('Voltage (V)')
    ax.set_ylabel('Absolute Current (A)')
    ax.set_yscale('log')
    plt.show()
    time.sleep(1)
    m2461.write(f"smu.source.output = 1")
   
    # Measurement loop
    for i in range(stepNum):
        current, voltage = measureM2461OnePointTSP(Vstart + i*Vstep,True,False)
        lstVoltage.append(voltage)
        lstCurrent.append(current)
        lstAbsCurrent.append(abs(current))
        update_plot(fig, ax, line,lstVoltage, lstAbsCurrent)
        #time.sleep(0.1)
    #Turn off the output
    m2461.write(f"smu.source.level = 0")
    m2461.write(f"smu.source.output = 0")
    dataToWrite = [['vRange: ', vRange]]
    dataToWrite.append(['iLimit: ', iCompliance])
    dataToWrite.append(['iRange: ', iRange[16:]])
    dataToWrite.append(['nplc: ', iNPLC, 'delay: ', delay])
    dataToWrite.append(['data Number:', stepNum])
    dataToWrite.append(['V_meas', 'I_meas'])

    for i in range(len(lstCurrent)):
        dataToWrite.append([lstVoltage[i], lstCurrent[i]])
    try:
        with open(filePath, 'w', newline = '') as file:
            writer = csv.writer(file)
            writer.writerows(dataToWrite)
            file.close()
    except IOError:
        print("Error: Could not open file for writing")
    plt.show()
    plt.savefig(os.path.splitext(filePath)[0] + '_log.png')
    ax.set_yscale('linear')
    plt.show()
    plt.savefig(os.path.splitext(filePath)[0] + '_lin.png')

In [18]:
def fncMeasureOneM2461SWETSP(arrVMeasurement, \
                  Sample = 'Test', sweepNum = 1,\
                  folderPath='250318\\Dev02\\Test',\
                  askToStart=False,\
                  terminal = "FRONT",\
                  vRange = "AUTO",\
                  iRange = "AUTO",\
                  iCompliance = 7,\
                  iMeasMode = "2WIRE",\
                  iNPLC = 1,\
                  delay = 0.1):
    lstCurrent = []
    lstVoltage = []
    lstAbsCurrent = []
    lstCalVoltage = []

    Vstart, Vstop, Vstep = arrVMeasurement
    fileName = "Sweep"+str(sweepNum)+Sample+'Star'+str(Vstart).replace("-","M")+'_Stop'+str(Vstop).replace("-","M")+'_Step'+str(Vstep).replace("-", "M")+'.csv'
    filePath = os.path.join(folderPath, fileName)
    print(filePath)
    stepNum = np.floor(abs((Vstart - Vstop)/Vstep)).astype(int) + 1
    
    setM2461IVMeasurementTSP(terminal, vRange, iRange, iCompliance, iMeasMode, iNPLC, delay)
    m2461.write(f'smu.source.sweeplinear("VLinSweep", {Vstart:0.2f}, {Vstop:0.2f}, {stepNum:0.2f}, {delay:0.2f})')
    m2461.write(f'trigger.model.initiate()')
    m2461.write(f'waitcomplete()')
    
    
    m2461.write(f'printbuffer(1, {stepNum:f}, defbuffer1.sourcevalues, defbuffer1.readings)')
    m2461.write(f'waitcomplete()')
    ## ensure that the measurement completes before retrieving data
    time.sleep((delay+0.15)*stepNum) # 
    ## ensure that the measurement completes before retrieving data
    data = m2461.read().split(', ') # this happens ansychonously with the sweep.
    print('Measured data: ', data)
    lstCurrent = [float(i) for i in data[1::2]] # get all odd elements, which are currents

    lstVoltage = [float(i) for i in data[0::2]] # get all even elements, which are voltages
    print (f'len current: {len(lstCurrent)}')
    print (f'len current: {len(lstVoltage)}')
    if Vstart > Vstop:
        Vstep = -abs(Vstep)
    else:
        Vstep = abs(Vstep)
    

    lstAbsCurrent = np.abs(lstCurrent)
    lstCalVoltage = [(Vstart + i*Vstep) for i in range(stepNum)]
    dataToWrite = [['vRange', vRange]]
    dataToWrite.append(['iCompliance', iCompliance])
    #dataToWrite.append(['iRange', iRange[16:]])
    dataToWrite.append(['iNPLC', iNPLC])
    dataToWrite.append(['data Number', stepNum])
    dataToWrite = [['V', 'I', 'V_cal','abs_I']]
    for i in range(len(lstCurrent)):
        dataToWrite.append([lstVoltage[i], lstCurrent[i], lstCalVoltage[i] , lstAbsCurrent[i]])
    try:
        with open(filePath, 'w', newline = '') as file:
            writer = csv.writer(file)
            writer.writerows(dataToWrite)
            file.close()
    except IOError:
        print("Error: Could not open file for writing")

    fig, ax = plt.subplots()
    line, = ax.plot(lstVoltage, lstAbsCurrent, 'o-')
    ax.set_xlabel('Voltage (V)')
    ax.set_ylabel('Current (A)')
    ax.set_yscale('linear')
    plt.show()
    plt.savefig(os.path.splitext(filePath)[0] + '_lin.png')
    ax.set_yscale('log')
    plt.show()
    plt.savefig(os.path.splitext(filePath)[0] + '_log.png')    


In [14]:
def fncMeasureOneM2461SWE(arrVMeasurement, \
                  Sample = 'Test', sweepNum = 1,\
                  folderPath='250318\\Dev02\\Test',\
                  askToStart=False):
    Vstart, Vstop, Vstep = arrVMeasurement
    fileName = "Sweep"+str(sweepNum)+Sample+'Star'+str(Vstart).replace("-","M")+'_Stop'+str(Vstop).replace("-","M")+'_Step'+str(Vstep).replace("-", "M")+'.csv'
    filePath = os.path.join(folderPath, fileName)
    print(filePath)
    stepNum = np.floor(abs((Vstart - Vstop)/Vstep)).astype(int) + 1
    vRange = "AUTO"
    iCompliance = 7
    iNPLC = 1
    delay = 0.1
    # data = m2461.query_ascii_values('FETC?')
    # print('this is the data 1', data)
    m2461.write('*RST')
    m2461.write("TRAC:CLE")
    m2461.write(':ROUT:TERM FRON')
    m2461.write(':SOUR:FUNC VOLT')
    # vRange = ':SOUR:VOLT:RANG:' + vRange
    # print(vRange)
    m2461.write(':SOUR:VOLT:RANG:AUTO ON')
    m2461.write('SOUR:VOLT:ILIM %0.2f' % iCompliance)
    m2461.write(':SENS:FUNC "CURR"')
    m2461.write('SENS:CURR:RANG:AUTO ON')
    m2461.write(':SENSE:CURR:NPLC %f' % iNPLC )
    m2461.write('SOUR:SWE:VOLT:LIN %0.2f, %0.2f, %f, %0.2f' % (Vstart, Vstop, stepNum, delay))
    m2461.write('INIT')
    #m2461.write('*WAI')
    m2461.write('*OPC')
    m2461.write('TRAC:DATA? 1, %f, "defbuffer1", SOUR, READ' % stepNum)
    #m2461.write('*WAI')
    m2461.write('*OPC')
    data = m2461.read().strip().split(',')
    
    #data = m2461.query_ascii_values('FETC?')
    
    print('Measured data: ', data)
   
    lstCurrent = [ float(i) for i in data[1::2]]# get all odd elements, which are currents

    lstVoltage = [float(v) for v in data[0::2]] # get all even elements, which are voltages

    # if Vstart > Vstop:
    #     Vstep = -abs(Vstep)
    # else:
    #     Vstep = abs(Vstep)
    lstAbsCurrent = [abs(i) for i in lstCurrent]
    lstCalVoltage = [(Vstart + i*Vstep) for i in range(stepNum)]
    dataToWrite = [['vRange', vRange]]
    dataToWrite.append(['iCompliance', iCompliance])
    #dataToWrite.append(['iRange', iRange[16:]])
    dataToWrite.append(['iNPLC', iNPLC])
    dataToWrite.append(['data Number', stepNum])
    dataToWrite = [['V', 'I', 'V_cal','abs_I']]
    for i in range(len(lstCurrent)):
        dataToWrite.append([lstVoltage[i], lstCurrent[i], lstCalVoltage[i] , lstAbsCurrent[i]])
    try:
        with open(filePath, 'w', newline = '') as file:
            writer = csv.writer(file)
            writer.writerows(dataToWrite)
            file.close()
    except IOError:
        print("Error: Could not open file for writing")

    fig, ax = plt.subplots()
    line, = ax.plot(lstVoltage, lstAbsCurrent, 'o-')
    ax.set_xlabel('Voltage (V)')
    ax.set_ylabel('Current (A)')
    ax.set_yscale('linear')
    plt.show()
    plt.savefig(os.path.splitext(filePath)[0] + '_lin.png')
    ax.set_yscale('log')
    plt.show()
    plt.savefig(os.path.splitext(filePath)[0] + '_log.png')
    
    
    
    

In [30]:
#run Keithley 2400 
arrKLVMeasMulti =["M2461SWETSP",[[0, 3,0.01],\
                            #[0, 3, 0.01],\
                            [0, -3, 0.01]
                            ]]
fncMeasureMultiple(arrKLVMeasMulti,sample, 27, folderPath)

../Devices/250328_Fab241201_IrOxNewSM\Dev08\H02\Sweep27H02Star0_Stop3_Step0.01.csv
Number of data points in buffer: 0
Measured data:  ['1.561245881e-06', '-9.7355110085e-12', '0.010001233779', '-2.4080314132e-11', '0.020000901073', '-2.063752641e-11', '0.030001148582', '-1.0309288145e-11', '0.040000461042', '-2.5227826772e-11', '0.049999557436', '-2.408025862e-11', '0.059999011457', '-1.5473407278e-11', '0.070000611246', '-1.8342320718e-11', '0.07999856025', '-1.6620947674e-11', '0.089998558164', '-1.4325783615e-11', '0.099997788668', '-9.7354832529e-12', '0.10999597609', '-2.0063721518e-11', '0.11999896169', '-1.3752020356e-11', '0.12999817729', '-8.014096331e-12', '0.1399987787', '-1.2030647312e-11', '0.14999875426', '-2.2761723062e-12', '0.15999889374', '-9.7354693751e-12', '0.17000152171', '-6.2926955313e-12', '0.18000119925', '-1.2030633434e-11', '0.19000220299', '-1.1456800786e-11', '0.20000192523', '-7.4402914385e-12', '0.21000237763', '-1.5473379522e-11', '0.22001813352', '-1.3

In [31]:
rm.close()

### REFERENCES

In [26]:
iLimit = 7
#m2461.write(f"smu.source.ilimit = %f " % iLimit)
m2461.write(f"smu.source.ilimit.level = %f " % iLimit)

37

In [13]:
vRange = "smu.ON"
m2461.write(f"smu.source.autorange = {vRange}") 

31

In [ ]:
m2461.write(f"beeper.beep(1,100)")
m2461.write(f"reset()")
m2461.write(f"status.reset()")
m2461.write(f"errorqueue.clear()")
# Configure source function as 2W DCVOLTS
m2461.write(f"smu.source.func = smu.FUNC_DC_VOLTAGE")
m2461.write(f"smu.measure.sense = smu.SENSE_2WIRE")
m2461.write(f"display.changescreen(display.SCREEN_HOME)")

43

In [ ]:
m2461.write(f'while defbuffer1.n < {stepNum} do delay(0.1) end')
data = m2461.query_ascii_values(f'printbuffer(1, {stepNum:f}, defbuffer1.sourcevalues, defbuffer1.readings)')
print(f'data: {data}')

m2461.write(f'print(1)')  # Send confirmation signal

In [17]:
setV = 1
outputON = False
m2461.write(f"smu.measure.func = smu.FUNC_DC_CURRENT") 
m2461.write(f"smu.source.func = smu.FUNC_DC_VOLTAGE")
m2461.write(f"smu.source.level = %f" %setV)
if (not outputON):
    m2461.write(f"smu.source.output = 1")
m2461.write(f"print(smu.measure.read())") 
rawData = m2461.read()
# print("%g\t%g\t%g", smua.nvbuffer1.timestamps[1], smua.nvbuffer1.readings[1], smua.nvbuffer2.readings[1])
arrRawData = rawData.split(",")
print ("rawData: ", rawData)

rawData:  -7.4574048325e-11

